In [1]:
from abc2xml import *
import regex as re

In [2]:
abcstring = expand_abc_include(readfile("Baroque_Vivaldi_Antonio_Art_Song.abc"))

In [3]:
abcstring

'X:1\n%%score 1 { ( 2 5 6 ) | ( 3 4 ) }\nL:1/8\nQ:1/4=86\nM:2/4\nK:F\nV:1 treble nm="Singstimme\\nVoice"\nV:2 treble nm="Pianoforte"\nV:5 treble \nV:6 treble \nV:3 bass \nV:4 bass \n[V:1]"^I. Andante" z|[V:2]!p! a/f/|[V:3]z1|[V:4]x1|[V:5]x1|[V:6]x1|\n[V:1]z4|[V:2]cccd|[V:3]z [FA][FA][FB]|[V:4]x4|[V:5]x4|[V:6]x4|\n[V:1]z4|[V:2](d2 e) b/g/|[V:3]B3 z|[V:4](F2 G) x|[V:5]x4|[V:6]x4|\n[V:1]z4|[V:2]eeef|[V:3]z [CGB][CGB][CFA]|[V:4]x4|[V:5]x4|[V:6]x4|\n[V:1]z4|[V:2](f2 g) (c\'/4a/4c\'/4a/4|[V:3](A2 G) z|[V:4]C3 x|[V:5]x4|[V:6]x4|\n[V:1]z4|[V:2].f) (c\'/4a/4c\'/4a/4 .f) (d\'/4b/4d\'/4b/4|[V:3][A,C] z [B,D] z|[V:4]x4|[V:5]x4|[V:6]x4|\n[V:1]z4|[V:2].g) (d\'/4b/4d\'/4b/4 .g) (c\'/4a/4c\'/4a/4|[V:3][B,E] z [A,F] z|[V:4]x4|[V:5]x4|[V:6]x4|\n[V:1]z4|[V:2].f) (c\'/4a/4c\'/4a/4 .e) (b/4g/4b/4g/4|[V:3][CA] z [CG] z|[V:4]x4|[V:5]x4|[V:6]x4|\n[V:1]z2 z c|[V:2].f) ([Ac]/[GB]/ [FA]) z|[V:3][F,F]2 z2|[V:4]x4|[V:5]x4|[V:6]x4|\n[V:1]c3/2 F/ F G|[V:2]z [A,CF][A,CF][B,DF]|[V:3][F,,F,]4-|[V:4]x4|[V:5]x4|[V:6]x4|\

In [4]:
fragments = re.split (r'^\s*X:', abcstring, flags=re.M)
preamble = fragments [0]    # tunes can be preceeded by formatting instructions
tunes = fragments[1:]

In [5]:
skip=0
num=1
xml_docs = []
rOpt=False
bOpt=False
fOpt=False
abctext = expand_abc_include (abcstring)
fragments = re.split (r'^\s*X:', abctext, flags=re.M)
preamble = fragments [0]    # tunes can be preceeded by formatting instructions
tunes = fragments[1:]
if not tunes and preamble: tunes, preamble = ['1\n' + preamble], ''  # tune without X:
for itune, tune in enumerate (tunes):
    if itune < skip: continue           # skip tunes, then read at most num tunes
    if itune >= skip + num: break
    tune = preamble + 'X:' + tune       # restore preamble before each tune
    try:                                # convert string abctext -> file pad/fnmNum.xml
        score = mxm.parse (tune, rOpt, bOpt, fOpt)
        ds = list (score.iter ('duration')) # need to iterate twice
        ss = [int (d.text) for d in ds]
        deler = reduce (ggd, ss + [21]) # greatest common divisor of all durations
        for i, d in enumerate (ds): d.text = str (ss [i] // deler)
        for d in score.iter ('divisions'): d.text = str (int (d.text) // deler)
        xml_docs.append (score)
    except ParseException:
        pass         # output already printed
    except Exception as err:
        info ('an exception occurred.\n%s' % err)

In [6]:
abctext = tune.replace ('[I:staff ','[I:staff')  # avoid false beam breaks
header, voices = splitHeaderVoices (abctext)

In [7]:
voices

[('1',
  '[V:1 treble nm="Singstimme\\nVoice"][V:1]"^I. Andante" z|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z2 z c|[V:1]c3/2 F/ F G|[V:1]G2 A A|[V:1]G A B c|[V:1]{A} G2 z c|[V:1]c3/2 F/ F G|[V:1]G2 A A|[V:1](A/G/) (f/d/) c =B|[V:1]c2 z2|[V:1]z4|[V:1]z2 z G|[V:1]G3/2 G/ G A/=B/|[V:1]c2 z2|[V:1]d2 c/=B/ A/G/|[V:1]{G} c3 c|[V:1]{e} d3 c/=B/|[V:1]{d} c2 z2|[V:1]d2 c/=B/ A/G/|[V:1]_e3 c|[V:1]=B3/2 B/ B B|[V:1]c2 z2|[V:1]z2 z c|[V:1]c3/2 c/ c _e|[V:1]_e2 d c|[V:1]B A/G/ d ^F|[V:1]G2 z B|[V:1]B3/2 B/ B d|[V:1]d2 c B|[V:1]A G/F/ c E|[V:1]F2 z A|[V:1]A3/2 A/ A d|[V:1]{d} ^c2 z c|[V:1]{e} d3/2 ^c/ d e|[V:1]{e} f2 z d|[V:1]f e d c|[V:1]{c} =B2 z B|[V:1]{d} c3/2 =B/ c d|[V:1]{d} e2 z c|[V:1]_e3/2 e/ e e|[V:1]_e2 d d|[V:1]c B/A/ G c|[V:1](c>f) c c|[V:1]{e} d (c/B/) A G|[V:1]{G} A2 z c|[V:1]_e3/2 e/ e e|[V:1]_e2 d d|[V:1]c B/A/ G c|[V:1](c>f) c c|[V:1]{e} d (c/B/) A G|[V:1]F2 z2|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]!fermata!z4|]'),
 ('2',
  '[V:2 treble nm="Pianoforte"][V

In [23]:
import re
from collections import defaultdict, Counter
from typing import List, Tuple, Dict, Optional, Literal

NOTE_BASE = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
IntervalMode = Literal["chromatic", "diatonic", "step_skip_leap", "contour"]


def strip_voice_and_text(abc: str) -> str:
    abc = re.sub(r'\".*?\"', '', abc)
    abc = re.sub(r'\[V:[^\]]*\]', '', abc)
    return abc

def abc_to_pitches_with_bars(abc: str) -> Tuple[List[int], List[int]]:
    """
    Returns:
      pitches: list of absolute pitches (semitones)
      note_to_bar: list mapping each pitch to its bar index
    """
    s = strip_voice_and_text(abc)
    pitches: List[int] = []
    note_to_bar: List[int] = []

    i = 0
    n = len(s)
    bar_idx = 0

    while i < n:
        ch = s[i]

        # barlines increment bar index
        if ch == '|':
            bar_idx += 1
            i += 1
            continue

        # skip structural characters
        if ch in '{}[]() \n\t\r':
            i += 1
            continue

        # rests
        if ch == 'z':
            i += 1
            while i < n and s[i] in '0123456789/':
                i += 1
            continue

        # accidentals
        acc = 0
        while i < n and s[i] in '^_=':
            if s[i] == '^':
                acc += 1
            elif s[i] == '_':
                acc -= 1
            i += 1

        if i >= n:
            break

        ch = s[i]
        if ch in 'ABCDEFGabcdefg':
            base = NOTE_BASE[ch.upper()]
            octave = 12 if ch.islower() else 0
            pitch = base + octave + acc
            i += 1

            # octave marks
            while i < n and s[i] in "',":
                pitch += 12 if s[i] == "'" else -12
                i += 1

            pitches.append(pitch)
            note_to_bar.append(bar_idx)

            # skip duration tokens
            while i < n and s[i] in '0123456789/':
                i += 1
        else:
            i += 1

    return pitches, note_to_bar

def motif_pattern_relative_to_first(window: List[int]) -> Tuple[int, ...]:
    p0 = window[0]
    return tuple(p - p0 for p in window)

In [24]:

def _sign(x: int) -> int:
    return 0 if x == 0 else (1 if x > 0 else -1)

def _chrom_to_diatonic_number(semitones: int) -> int:
    """
    Map chromatic interval size to diatonic interval number (quality-free).
    Uses nearest/common mapping:
      0->0
      1/2->1 (2nd)
      3/4->2 (3rd)
      5->3 (4th)
      6/7->4 (5th)   (tritone grouped with 5th here; tweak if you want)
      8/9->5 (6th)
      10/11->6 (7th)
      12->7 (8ve)
    and repeats every 12 with octave increments.
    """
    sgn = _sign(semitones)
    a = abs(semitones)
    octs, rem = divmod(a, 12)

    if rem == 0:
        base = 0
    elif rem in (1, 2):
        base = 1
    elif rem in (3, 4):
        base = 2
    elif rem == 5:
        base = 3
    elif rem in (6, 7):
        base = 4
    elif rem in (8, 9):
        base = 5
    else:  # 10,11
        base = 6

    return sgn * (base + 7 * octs)

def _coarsen_pattern(pat_chromatic: Tuple[int, ...], interval_mode: IntervalMode) -> Tuple[int, ...]:
    """
    Convert chromatic (semitone) offsets-from-first into a coarser signature.
    """
    if interval_mode == "chromatic":
        return pat_chromatic

    if interval_mode == "diatonic":
        return tuple(_chrom_to_diatonic_number(x) for x in pat_chromatic)

    if interval_mode == "step_skip_leap":
        # first convert to diatonic numbers, then bucket by size
        diat = [_chrom_to_diatonic_number(x) for x in pat_chromatic]
        out = [0]
        for d in diat[1:]:
            ad = abs(d)
            if ad == 0:
                cls = 0
            elif ad == 1:
                cls = 1      # step (2nd)
            elif ad == 2:
                cls = 2      # skip (3rd)
            else:
                cls = 3      # leap (4th+)
            out.append(_sign(d) * cls)
        return tuple(out)

    if interval_mode == "contour":
        return tuple(_sign(x) for x in pat_chromatic)

    raise ValueError(f"Unknown interval_mode: {interval_mode}")

def count_interval_motifs(
    pitches: List[int],
    note_to_bar: List[int],
    window_notes: int,
    stride_notes: int = 1,
    interval_mode: IntervalMode = "chromatic",
    bar_numbering: Literal["0-based", "1-based"] = "1-based",
) -> Tuple[Counter, Dict[Tuple[int, ...], List[dict]]]:
    """
    Always computes chromatic pattern relative-to-first,
    then coarsens it according to interval_mode for counting.
    """
    if window_notes <= 0:
        raise ValueError("window_notes must be >= 1")
    if stride_notes <= 0:
        raise ValueError("stride_notes must be >= 1")
    if len(pitches) != len(note_to_bar):
        raise ValueError("pitches and note_to_bar must have same length")

    counts = Counter()
    occurrences: Dict[Tuple[int, ...], List[dict]] = defaultdict(list)

    n = len(pitches)
    bar_offset = 1 if bar_numbering == "1-based" else 0

    for start in range(0, n - window_notes + 1, stride_notes):
        window = pitches[start:start + window_notes]
        pat_chromatic = motif_pattern_relative_to_first(window)  # <-- unchanged
        pat = _coarsen_pattern(pat_chromatic, interval_mode)

        counts[pat] += 1
        occurrences[pat].append({
            "start_note": start,
            "start_bar": note_to_bar[start] + bar_offset,
            # If you also want: original chromatic pattern for inspection:
            "chromatic_pattern": pat_chromatic if interval_mode != "chromatic" else None
        })

    return counts, occurrences

def find_top_motif(
    abc: str,
    window_notes: int,
    stride_notes: int = 1,
    min_count: int = 2,
    interval_mode: IntervalMode = "chromatic",
    bar_numbering: Literal["0-based", "1-based"] = "1-based",
):
    pitches, note_to_bar = abc_to_pitches_with_bars(abc)
    counts, occurrences = count_interval_motifs(
        pitches,
        note_to_bar,
        window_notes=window_notes,
        stride_notes=stride_notes,
        interval_mode=interval_mode,
        bar_numbering=bar_numbering,
    )

    best_pattern: Optional[Tuple[int, ...]] = None
    best_count = 0
    for pat, c in counts.most_common():
        if c >= min_count:
            best_pattern, best_count = pat, c
            break

    return {
        "total_notes": len(pitches),
        "interval_mode": interval_mode,
        "window_notes": window_notes,
        "stride_notes": stride_notes,
        "best_pattern": best_pattern,
        "best_count": best_count,
        "occurrences": occurrences.get(best_pattern, []) if best_pattern is not None else [],
    }

In [21]:
print(abc_voice)

[V:1 treble nm="Singstimme\nVoice"][V:1]"^I. Andante" z|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z2 z c|[V:1]c3/2 F/ F G|[V:1]G2 A A|[V:1]G A B c|[V:1]{A} G2 z c|[V:1]c3/2 F/ F G|[V:1]G2 A A|[V:1](A/G/) (f/d/) c =B|[V:1]c2 z2|[V:1]z4|[V:1]z2 z G|[V:1]G3/2 G/ G A/=B/|[V:1]c2 z2|[V:1]d2 c/=B/ A/G/|[V:1]{G} c3 c|[V:1]{e} d3 c/=B/|[V:1]{d} c2 z2|[V:1]d2 c/=B/ A/G/|[V:1]_e3 c|[V:1]=B3/2 B/ B B|[V:1]c2 z2|[V:1]z2 z c|[V:1]c3/2 c/ c _e|[V:1]_e2 d c|[V:1]B A/G/ d ^F|[V:1]G2 z B|[V:1]B3/2 B/ B d|[V:1]d2 c B|[V:1]A G/F/ c E|[V:1]F2 z A|[V:1]A3/2 A/ A d|[V:1]{d} ^c2 z c|[V:1]{e} d3/2 ^c/ d e|[V:1]{e} f2 z d|[V:1]f e d c|[V:1]{c} =B2 z B|[V:1]{d} c3/2 =B/ c d|[V:1]{d} e2 z c|[V:1]_e3/2 e/ e e|[V:1]_e2 d d|[V:1]c B/A/ G c|[V:1](c>f) c c|[V:1]{e} d (c/B/) A G|[V:1]{G} A2 z c|[V:1]_e3/2 e/ e e|[V:1]_e2 d d|[V:1]c B/A/ G c|[V:1](c>f) c c|[V:1]{e} d (c/B/) A G|[V:1]F2 z2|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]z4|[V:1]!fermata!z4|]


In [28]:
abc_voice = voices[0][1]
print(find_top_motif(abc_voice, interval_mode= 'contour', window_notes=5, stride_notes=1, min_count=2))


{'total_notes': 173, 'interval_mode': 'contour', 'window_notes': 5, 'stride_notes': 1, 'best_pattern': (0, -1, -1, -1, -1), 'best_count': 35, 'occurrences': [{'start_note': 1, 'start_bar': 10, 'chromatic_pattern': (0, -7, -7, -5, -5)}, {'start_note': 15, 'start_bar': 14, 'chromatic_pattern': (0, -7, -7, -5, -5)}, {'start_note': 24, 'start_bar': 16, 'chromatic_pattern': (0, -3, -5, -6, -5)}, {'start_note': 25, 'start_bar': 16, 'chromatic_pattern': (0, -2, -3, -2, -7)}, {'start_note': 28, 'start_bar': 17, 'chromatic_pattern': (0, -5, -5, -5, -5)}, {'start_note': 36, 'start_bar': 22, 'chromatic_pattern': (0, -2, -3, -5, -7)}, {'start_note': 37, 'start_bar': 22, 'chromatic_pattern': (0, -1, -3, -5, -5)}, {'start_note': 44, 'start_bar': 24, 'chromatic_pattern': (0, -2, -4, -5, -2)}, {'start_note': 50, 'start_bar': 26, 'chromatic_pattern': (0, -2, -3, -5, -7)}, {'start_note': 55, 'start_bar': 27, 'chromatic_pattern': (0, -3, -4, -4, -4)}, {'start_note': 56, 'start_bar': 27, 'chromatic_patter